In [24]:
# importing Libraries and Setting auxiliary Variables


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os


In [25]:
def load_image(path, img_size=(64,64)): 
    images = []
    labels = []
    for label, class_dir in enumerate( ['cat' , 'dog'] ):
        class_path = os.path.join(path, class_dir)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = Image.open(img_path).convert('L').resize(img_size)
            images.append(np.array(img).flatten())
            labels.append(label)

    return np.array(images) , np.array(labels)


def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1- sigmoid(x))

class DenseLayer:
    def __init__(self, input_size, output_size): #input_size = no. of rows ------ # output = no. of output neurons
        self.weights = np.random.randn(input_size, output_size) * 0.1
        self.bias = np.zeros((1, output_size))


    def forward(self, inputs):
        self.input = inputs
        self.output = sigmoid(np.dot(inputs, self.weights) + self.bias )
        return self.output
    
    def backward(self, dvalues):
        self.dweights = np.dot(self.input.T, dvalues)
        self.dbias = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)
        return self.dinputs
    


class NeuralNetwork:
    def __init__(self, image_size_flatten):
        self.layer1 = DenseLayer(image_size_flatten, 128)
        self.layer2 = DenseLayer(128, 64)
        self.layer3 = DenseLayer(64,1)

    def forward(self, image_dataset_array):
        out = self.layer1.forward(image_dataset_array)              # self.inputs: (9, 4096)     self.output: (9, 128)  image_dataset_array: (9, 4096)
        out = self.layer2.forward(out)                              # self.inputs: (9, 128)      self.output: (9, 64)   image_dataset_array: (9, 128)
        out = self.layer3.forward(out)                              # self.inputs: (9, 64)       self.output: (9, 1)    image_dataset_array: (9, 64
        return out
    
    def backward(self, dvalues):
        dvalues = self.layer3.backward(dvalues)                     # self.dvalues: (9, 1)      self.dweights: (64, 1)      OUTPUT: (9, 64)
        dvalues = self.layer2.backward(dvalues)                     # self.dvalues: (9, 64)     self.dweights: (128, 64)    OUTPUT: (9, 128)
        self.layer1.backward(dvalues)                               # self.dvalues: (9, 128)    self.dweights: (4096, 128)  OUTPUT: (9, 4096)

    def update(self, learning_rate):
        self.layer1.weights -= learning_rate * self.layer1.dweights
        self.layer1.bias -= learning_rate * self.layer1.dbias
        
        self.layer2.weights -= learning_rate * self.layer2.dweights
        self.layer2.bias -= learning_rate * self.layer2.dbias
        
        self.layer3.weights -= learning_rate * self.layer3.dweights
        self.layer3.bias -= learning_rate * self.layer3.dbias


def train(model, image_dataset_array, actual_labels, epochs, learning_rate):
    for epoch in range(epochs):
        output = model.forward(image_dataset_array)
        loss = np.mean((output-actual_labels)**2 )
        print(f" -----> Epoch: {epoch} \n--------->Loss: {loss}")

        dvalues = 2* (output-actual_labels) / actual_labels.size
        model.backward(dvalues)
        model.update(learning_rate)



image_dataset_array, actual_labels = load_image('dataset')
actual_labels = actual_labels.reshape(-1, 1)

input_image_flat_size = image_dataset_array.shape[1]



model1 = NeuralNetwork(image_size_flatten=input_image_flat_size)


train(model1, image_dataset_array, actual_labels, epochs=300, learning_rate=0.01)

C:\Users\gchakshu\AppData\Local\Temp\ipykernel_3456\3745664846.py:17: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


 -----> Epoch: 0 
--------->Loss: 0.24253981670532043
 -----> Epoch: 1 
--------->Loss: 0.26379020186276547
 -----> Epoch: 2 
--------->Loss: 0.2952041468423206
 -----> Epoch: 3 
--------->Loss: 0.2689311553743247
 -----> Epoch: 4 
--------->Loss: 0.2328225494891022
 -----> Epoch: 5 
--------->Loss: 0.12277233105073877
 -----> Epoch: 6 
--------->Loss: 0.19154957230114256
 -----> Epoch: 7 
--------->Loss: 0.11126612533606431
 -----> Epoch: 8 
--------->Loss: 0.1740112291882966
 -----> Epoch: 9 
--------->Loss: 0.10224433909395349
 -----> Epoch: 10 
--------->Loss: 0.15265373397284973
 -----> Epoch: 11 
--------->Loss: 0.09391898599612557
 -----> Epoch: 12 
--------->Loss: 0.14621295120112363
 -----> Epoch: 13 
--------->Loss: 0.08793110070705056
 -----> Epoch: 14 
--------->Loss: 0.1306018436257649
 -----> Epoch: 15 
--------->Loss: 0.08222910639415298
 -----> Epoch: 16 
--------->Loss: 0.13711413843917813
 -----> Epoch: 17 
--------->Loss: 0.07697756790764586
 -----> Epoch: 18 
------

### Testing Model

In [26]:
def load_mixed_images(path, img_size=(64, 64)):
    images = []
    actual_labels = []
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = Image.open(img_path).convert('L').resize(img_size)
        images.append(np.array(img).flatten())
        label = 1 if 'dog' in img_name.lower() else 0  # Assuming naming convention indicates label
        actual_labels.append(label)
    return np.array(images), np.array(actual_labels)

# Define a function to calculate accuracy
def calculate_accuracy(predictions, actual_labels):
    correct_predictions = (predictions.flatten() > 0.5).astype(int) == actual_labels
    accuracy = np.mean(correct_predictions)
    return accuracy

# Define a function for precision, recall, and F1-score
def calculate_precision_recall_f1(predictions, actual_labels):
    predictions_binary = (predictions.flatten() > 0.5).astype(int)
    true_positives = np.sum((predictions_binary == 1) & (actual_labels == 1))
    false_positives = np.sum((predictions_binary == 1) & (actual_labels == 0))
    false_negatives = np.sum((predictions_binary == 0) & (actual_labels == 1))

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score

# Load the mixed dataset
X_mixed, y_actual = load_mixed_images('test_set')

# Get model predictions
predictions = model1.forward(X_mixed)

# Calculate accuracy
accuracy = calculate_accuracy(predictions, y_actual)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Calculate precision, recall, and F1-score
precision, recall, f1_score = calculate_precision_recall_f1(predictions, y_actual)
print(f"Number of images - {sum(os.path.isfile(os.path.join('test_set', f)) for f in os.listdir('test_set'))}")
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1_score:.2f}')

Accuracy: 40.00%
Number of images - 5
Precision: 0.40
Recall: 1.00
F1 Score: 0.57


C:\Users\gchakshu\AppData\Local\Temp\ipykernel_3456\3745664846.py:17: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
